In [1]:
import sys

sys.path.append('..')

In [2]:
from pathlib import Path

import numpy as np
from ipywidgets import interact
from PIL import Image, ImageDraw
from skimage import filters, feature, draw
from skimage import exposure as ex
from skimage import morphology as morph
from skimage import restoration as rest

from scipy.ndimage import interpolation as inter

import digi_leap.const as const
import digi_leap.util as util
from digi_leap.ocr_score import score_tesseract, score_easyocr

In [3]:
DATA_DIR = Path('..') / 'data'
LABELS_DIR = DATA_DIR / 'labels' / 'typewritten'
# LABELS_DIR = DATA_DIR / 'labels' / 'handwritten'

In [4]:
IMAGES = sorted(LABELS_DIR.glob('*.jpg'))

IDX = 3240   # A target image with plenty of underlines
# IDX = 46    # Lots of odd colors
# IDX = 2880  # Underlines
# IDX = 2939  # Rotated
# IDX = 28970  # Border

### Display the raw image

In [5]:
def raw_image(idx):
    path = IMAGES[idx]
    print(path)
    image = Image.open(path)
    display(image)


# raw_image(IDX)
# interact(raw_image, idx=(0, len(IMAGES) - 1))

### Find lines

In [6]:
WINDOW_SIZE = 11
K = 0.032


def show_lines(idx):
    image = Image.open(IMAGES[idx]).convert('L')

    binary = np.asarray(image).copy()
    threshold = filters.threshold_sauvola(binary, window_size=WINDOW_SIZE, k=K)
    binary = binary < threshold

    h_lines = util.find_lines(binary, const.NEAR_HORIZ, line_length=100, line_gap=5)

    image = image.convert('RGB')
    draw = ImageDraw.Draw(image)

    for line in h_lines:
        draw.line(line, width=1, fill=(255, 0, 0))

    display(image)


# show_lines(IDX)
# interact(show_lines, idx=(0, len(IMAGES) - 1))

In [7]:
WINDOW_SIZE = 11
K = 0.032


def detect(idx):
    image = Image.open(IMAGES[idx]).convert('L')

    score0 = score_tesseract(image)

    adjusted = np.asarray(image).copy()
    adjusted = filters.rank.modal(adjusted, selem=morph.disk(2))
#     adjusted = ex.adjust_gamma(adjusted, gamma=2)
#     adjusted = ex.rescale_intensity(adjusted)

#     threshold = filters.threshold_sauvola(adjusted, window_size=WINDOW_SIZE, k=K)
#     binary = adjusted < threshold

#     binary = morph.binary_closing(binary, selem=morph.disk(2))

#     h_lines = util.find_lines(binary, const.NEAR_HORIZ, line_length=100, line_gap=5)

#     mask = np.zeros_like(binary)
#     for (x0, y0), (x1, y1) in h_lines:
#         rr, cc = draw.line(y0, x0, y1, x1)
#         mask[rr, cc] = True
#     mask = morph.binary_dilation(mask, selem=morph.rectangle(12, 40))

#     painted = rest.inpaint_biharmonic(np.asarray(adjusted).copy(), mask)

    out_image = util.to_pil(adjusted).convert('L')

#     print(score0)
#     print(score0.text)

    score = score_tesseract(out_image)
    print(score)
    print(score.text)

    display(out_image)


# detect(IDX)
# interact(detect, idx=(0, len(IMAGES) - 1))

In [8]:
%%javascript
$('.output_label').css('font-size', '16pt')

<IPython.core.display.Javascript object>